# Análise exploratória de dados

In [2]:
import pandas as pd

## 1. Descrição dos dados

In [3]:
dictionary = pd.read_csv('../data/external/data-dict.csv')
dictionary

,variavel,descricao,tipo,subtipo
0,total_bill,Valor total da conta (em dólares),quantitativa,contínua
1,tip,Valor da gorjeta (em dólares),quantitativa,contínua
2,sex,Sexo da pessoa que pagou a conta,qualitativa,nominal
3,smoker,Indica se havia fumantes no grupo,qualitativa,nominal
4,day,Dia da semana em que a refeição foi consumida,qualitativa,nominal
5,time,Momento do dia em que a refeição foi consumida,qualitativa,nominal
6,size,Número de pessoas no grupo,quantitativa,discreta


## 2. Perguntas de partida e hipóteses

 ### Perguntas de partida
 - O valor da gorjeta é proporcional ao valor da conta?
 - Ter um fumante no grupo muda no valor da gorjeta?
 - Quais dias tiveram mais pessoas?
 - Quais refeições tiveram mais pessoas?
 - Qual média de gorjeta e conta por dia? E por refeição?
 - A hora impacta no valor da gorjeta?
 - A quantidade de pessoas no grupo impacta no valor da gorjeta?
 
 ### Hipóteses
 
 - A quantidade de pessoas interfere no valor da conta e da gorjeta
 - Os finais 

## 3. Insights